# Training a classifier for weights of author vision components

The features are the distance of the author to any tweet in the conversation
indicated by the following structures:
- subtree to viewed tweet from a tweet the author wrote
- root closeness of viewed tweet
- time delta to viewed tweet from tweets the author wrote


In [69]:
import pandas as pd

df = pd.read_pickle("data/vision_graph_data.pkl")

df.describe()

,reply_distance_2,reply_distance_3,timedelta,root_distance_0,root_distance_1,root_distance_2,y,current,beam_node,has_followed_path,has_follow_path
count,7.418172e+06,7.418172e+06,7.418172e+06,7.418172e+06,7.418172e+06,7.418172e+06,7.418172e+06,7.418172e+06,7.418172e+06,7418172.0,7418172.0
mean,1.136304e-02,6.845352e-04,3.175986e+04,1.017137e-02,9.018925e-01,3.807852e-02,1.549290e-02,1.448474e+18,1.448347e+18,0.0,0.0
std,1.059902e-01,2.615467e-02,3.023372e+05,1.003390e-01,2.974599e-01,1.913859e-01,1.235025e-01,3.204302e+17,3.203389e+17,0.0,0.0
min,0.000000e+00,0.000000e+00,1.000000e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.881900e+04,4.881900e+04,0.0,0.0
25%,0.000000e+00,0.000000e+00,3.462000e+03,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.509801e+18,1.509693e+18,0.0,0.0
50%,0.000000e+00,0.000000e+00,1.197000e+04,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.518589e+18,1.518371e+18,0.0,0.0
75%,0.000000e+00,0.000000e+00,3.445400e+04,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.524345e+18,1.524051e+18,0.0,0.0
max,1.000000e+00,1.000000e+00,1.103033e+08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,7.694580e+18,7.694580e+18,0.0,0.0


In [70]:
# remove non-features
current = df.current
beam_node = df.beam_node

df = df.drop(["current", "beam_node"], axis=1)

# normalize timedelta (put between 0 and 1)
dt = df.timedelta
timedelta_normalized = (dt - dt.min()) / (dt.max() - dt.min())
df.timedelta = timedelta_normalized

df

,reply_distance_2,reply_distance_3,timedelta,root_distance_0,root_distance_1,root_distance_2,y,has_followed_path,has_follow_path
0,0,0,0.000000e+00,1,0,0,0,0,0
1,0,0,1.813182e-14,1,0,0,0,0,0
2,0,0,9.065911e-15,0,0,0,0,0,0
3,0,0,2.719773e-14,1,0,0,0,0,0
4,0,0,1.813182e-14,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
7418167,0,0,9.174702e-06,0,1,0,0,0,0
7418168,0,0,7.978002e-06,0,1,0,0,0,0
7418169,0,0,7.660695e-06,0,1,0,0,0,0
7418170,0,0,5.394217e-06,0,0,1,0,0,0


In [71]:
from sklearn.model_selection import train_test_split

# selecting train and test datasets
train, test = train_test_split(df, test_size=0.2)
train.describe()


,reply_distance_2,reply_distance_3,timedelta,root_distance_0,root_distance_1,root_distance_2,y,has_followed_path,has_follow_path
count,5.934537e+06,5.934537e+06,5.934537e+06,5.934537e+06,5.934537e+06,5.934537e+06,5.934537e+06,5934537.0,5934537.0
mean,1.137797e-02,6.821088e-04,2.883929e-04,1.018058e-02,9.018875e-01,3.809514e-02,1.549860e-02,0.0,0.0
std,1.060590e-01,2.610831e-02,2.766746e-03,1.003839e-01,2.974667e-01,1.914260e-01,1.235249e-01,0.0,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
25%,0.000000e+00,0.000000e+00,3.139525e-05,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
50%,0.000000e+00,0.000000e+00,1.085824e-04,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
75%,0.000000e+00,0.000000e+00,3.124294e-04,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0,0.0


In [72]:
# train the model
y = train.y
x = train.drop("y", axis=1)
print(x.shape)
print(y.shape)


(5934537, 8)
(5934537,)


In [73]:
# convert train dataset to numpy arrays not sure this is neded
#x = x.to_numpy()
#y = y.to_numpy()
#print(x[:5])
#print(y[:5])

In [74]:
# import tensorflow and train the model
import tensorflow as tf

print(tf.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

input_shape = (x.shape[1],)
model = Sequential([
    Dense(1, activation='sigmoid', input_shape=input_shape)
])

# stachistical gradient descend as a classifier seem appropriate
model.compile(
    optimizer='sgd',
    loss='binary_crossentropy',
    metrics=['accuracy', 'mae']
)

model.fit(x, y)


2.6.0
185455/185455 [==============================] - 218s 1ms/step - loss: 0.0342 - accuracy: 0.9925 - mae: 0.0160    

In [75]:
# evaluate the model on the test set
test_y = test.y
test_x = test.drop("y", axis=1)

In [76]:
# convert train dataset to numpy arrays not sure this is neded
# test_x = test_x.to_numpy()
# test_y = test_y.to_numpy()

In [77]:
loss, accuracy, mae = model.evaluate(test_x, test_y)
print("the accuracy on the training set is {} and the mae is {}".format(accuracy, mae))

46364/46364 [==============================] - 45s 964us/step - loss: 0.0306 - accuracy: 0.9936 - mae: 0.0120

In [79]:
# have a look at some prediction
reply_distance_2 = test[test["reply_distance_2"] == 1]
reply_distance_2 = reply_distance_2.drop("y", axis=1)
first_rows = reply_distance_2.head(2)
print(first_rows)
model.predict(first_rows)

         reply_distance_2  reply_distance_3  timedelta  root_distance_0  \
7343908                 1                 0   0.000019                0   
6154569                 1                 0   0.000017                1   

         root_distance_1  root_distance_2  has_followed_path  has_follow_path  
7343908                0                1                  0                0  
6154569                0                0                  0                0  


array([[0.5600261],
       [0.9601488]], dtype=float32)

By intuition, this result makes sense:
- In the first example the beam node is two removed from the root where in the second
example the it is the root node itself.
- Also the time elapsed is shorted (the time delta is normalized so how much time that is I am not sure

For these reasons ...
 - the second sample has a probability of 96% of having been seen
 - the first sample only has a probability of 56% of having been seen (which is still high)

In [83]:
# let's have a look at the weights and biases of the hidden layer

first_layer_weights = model.layers[0].get_weights()[0]
first_layer_biases  = model.layers[0].get_weights()[1]
print(first_layer_weights)
column_names = x.columns.values
for i in range(len(column_names)):
    print("feature {} has weight {} \n" .format(column_names[i], first_layer_weights[i]))



[[ 4.049399  ]
 [ 0.5441881 ]
 [ 0.7265376 ]
 [ 2.231453  ]
 [-2.820653  ]
 [-0.7092159 ]
 [ 0.10275137]
 [-0.47634667]]
feature reply_distance_2 has weight [4.049399] 

feature reply_distance_3 has weight [0.5441881] 

feature timedelta has weight [0.7265376] 

feature root_distance_0 has weight [2.231453] 

feature root_distance_1 has weight [-2.820653] 

feature root_distance_2 has weight [-0.7092159] 

feature has_followed_path has weight [0.10275137] 

feature has_follow_path has weight [-0.47634667] 

